### Topics:

    Constraints :  PRIMARY KEY, FOREIGN KEY, NOT NULL, UNIQUE, CHECK
    Expressions : CASE, COALESCE, NULLIF

### PRIMARY KEY

A primary key is a column or a group of columns that uniquely identifies each row in a table. You create a primary key for a table by using the PRIMARY KEY constraint.

If the primary key consists of only one column, you can define use PRIMARY KEY constraint as a column constraint:

    CREATE TABLE table_name (
        pk_column data_type PRIMARY KEY,
        ...
    );

In case the primary key has two or more columns, you must use the PRIMARY KEY constraint as a table constraint:

    CREATE TABLE table_name (
        pk_column_1 data_type,
        pk_column_2 data type,
        ...
        PRIMARY KEY (pk_column_1, pk_column_2)
    );

Each table can contain only one primary key. All columns that participate in the primary key must be defined as NOT NULL. SQL Server automatically sets the NOT NULL constraint for all the primary key columns if the NOT NULL constraint is not specified for these columns.

SQL Server also automatically creates a unique clustered index (or a non-clustered index if specified as such) when you create a primary key.

In [11]:
import pyodbc
import os
import pandas as pd

#Check if drivers are installed
#[x for x in pyodbc.drivers() if x.startswith("Microsoft Access Driver")]

# Define the connection string
conn_str = (
    r'DRIVER={ODBC Driver 17 for SQL Server};'
    r'SERVER=localhost;'
    r'DATABASE=BikeStores;'
    r'Trusted_Connection=yes;'
)

# Establish the connection
conn = pyodbc.connect(conn_str, autocommit=True)

# Create a cursor
cursor = conn.cursor()

To create a table with a single primary key define the PRIMARY KEY constraint next to datatype

In [2]:
cursor.execute('''
CREATE TABLE sales.activities (
    activity_id INT PRIMARY KEY IDENTITY,
    activity_name VARCHAR (255) NOT NULL,
    activity_date DATE NOT NULL
);
''')

To create two columns as primary keys define PRIMARY KEY constraint at the end of statement

In [3]:
cursor.execute('''
CREATE TABLE sales.participants(
    activity_id int,
    customer_id int,
    PRIMARY KEY(activity_id, customer_id)
);
''')


But if you have a table with no primary key, then ALTER table to assign a primary key

In [4]:
cursor.execute('''
CREATE TABLE sales.events(
    event_id INT NOT NULL,
    event_name VARCHAR(255),
    start_date DATE NOT NULL,
    duration DEC(5,2)
);
''')

cursor.execute('''
ALTER TABLE sales.events 
ADD PRIMARY KEY(event_id);
''')

#### FOREIGN KEY

Lets create the following tables first:

In [18]:
cursor.execute('''
CREATE TABLE sales.vendor_groups (
    group_id INT IDENTITY PRIMARY KEY,
    group_name VARCHAR (100) NOT NULL
);

CREATE TABLE sales.vendors (
        vendor_id INT IDENTITY PRIMARY KEY,
        vendor_name VARCHAR(100) NOT NULL,
        group_id INT NOT NULL,
);
''')

Each vendor belongs to a vendor group and each vendor group may have zero or more vendors. The relationship between the vendor_groups and vendors tables is one-to-many.

For each row in the  vendors table, you can always find a corresponding row in the vendor_groups table

However, with the current tables setup, you can insert a row into the  vendors table without a corresponding row in the vendor_groups table. Similarly, you can also delete a row in the vendor_groups table without updating or deleting the corresponding rows in the  vendors table that results in orphaned rows in the  vendors table.

To enforce the link between data in the vendor_groups and vendors tables, you need to establish a foreign key in the vendors table.

A foreign key is a column or a group of columns in one table that uniquely identifies a row of another table (or the same table in case of self-reference).

To create a foreign key, you use the FOREIGN KEY constraint.

In [ ]:
cursor.execute('''DROP TABLE sales.vendors ''')

In [20]:
cursor.execute('''
CREATE TABLE sales.vendors (
        vendor_id INT IDENTITY PRIMARY KEY,
        vendor_name VARCHAR(100) NOT NULL,
        group_id INT NOT NULL,
        CONSTRAINT fk_group FOREIGN KEY (group_id) 
        REFERENCES sales.vendor_groups(group_id)
);
''')

The vendor_groups table now is called the parent table that is the table to which the foreign key constraint references. The vendors table is called the child table that is the table to which the foreign key constraint is applied.

In the statement above, the following clause creates a FOREIGN KEY constraint named fk_group that links the group_id in the  vendors table to the group_id in the vendor_groups table:

    CONSTRAINT fk_group FOREIGN KEY (group_id) REFERENCES sales.vendor_groups(group_id)


The general syntax for creating a FOREIGN KEY constraint is as follows:

    CONSTRAINT fk_constraint_name 
    FOREIGN KEY (column_1, column2,...)
    REFERENCES parent_table_name(column1,column2,..)

In [21]:
cursor.execute('''
INSERT INTO sales.vendor_groups(group_name)
VALUES('Third-Party Vendors'),
      ('Interco Vendors'),
      ('One-time Vendors');
''')

In [23]:
cursor.execute('''
INSERT INTO sales.vendors(vendor_name, group_id)
VALUES('ABC Corp',1);
''')

In [24]:
cursor.execute('''
INSERT INTO sales.vendors(vendor_name, group_id)
VALUES('XYZ Corp',4);
''')

IntegrityError: ('23000', '[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The INSERT statement conflicted with the FOREIGN KEY constraint "fk_group". The conflict occurred in database "BikeStores", table "sales.vendor_groups", column \'group_id\'. (547) (SQLExecDirectW); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The statement has been terminated. (3621)')

#### Referential actions
The foreign key constraint ensures referential integrity. It means that you can only insert a row into the child table if there is a corresponding row in the parent table.

Besides, the foreign key constraint allows you to define the referential actions when the row in the parent table is updated or deleted as follows:

    FOREIGN KEY (foreign_key_columns)
        REFERENCES parent_table(parent_key_columns)
        ON UPDATE action 
        ON DELETE action;

Code language: SQL (Structured Query Language) (sql)

The ON UPDATE and ON DELETE specify which action will execute when a row in the parent table is updated and deleted. The following are permitted actions : NO ACTION, CASCADE, SET NULL, and SET DEFAULT

#### Delete actions of rows in the parent table
If you delete one or more rows in the parent table, you can set one of the following actions:

    ON DELETE NO ACTION: SQL Server raises an error and rolls back the delete action on the row in the parent table.
    ON DELETE CASCADE: SQL Server deletes the rows in the child table that is corresponding to the row deleted from the parent table.
    ON DELETE SET NULL: SQL Server sets the rows in the child table to NULL if the corresponding rows in the parent table are deleted. To execute this action, the foreign key columns must be nullable.
    ON DELETE SET DEFAULT SQL Server sets the rows in the child table to their default values if the corresponding rows in the parent table are deleted. To execute this action, the foreign key columns must have default definitions. Note that a nullable column has a default value of NULL if no default value specified.
    By default, SQL Server appliesON DELETE NO ACTION if you don’t explicitly specify any action.

#### Update action of rows in the parent table
If you update one or more rows in the parent table, you can set one of the following actions:
    
    ON UPDATE NO ACTION: SQL Server raises an error and rolls back the update action on the row in the parent table.
    ON UPDATE CASCADE: SQL Server updates the corresponding rows in the child table when the rows in the parent table are updated.
    ON UPDATE SET NULL: SQL Server sets the rows in the child table to NULL when the corresponding row in the parent table is updated. Note that the foreign key columns must be nullable for this action to execute.
    ON UPDATE SET DEFAULT: SQL Server sets the default values for the rows in the child table that have the corresponding rows in the parent table updated.

In [25]:
conn.close()
conn.closed

True

### NOT NULL 

In [40]:
import pyodbc
import os
import pandas as pd

#Check if drivers are installed
#[x for x in pyodbc.drivers() if x.startswith("Microsoft Access Driver")]

# Define the connection string
conn_str = (
    r'DRIVER={ODBC Driver 17 for SQL Server};'
    r'SERVER=localhost;'
    r'DATABASE=hr;'
    r'Trusted_Connection=yes;'
)

# Establish the connection
conn = pyodbc.connect(conn_str, autocommit=True)

# Create a cursor
cursor = conn.cursor()

In [49]:
cursor.execute('''
CREATE TABLE hr.persons(
    person_id INT IDENTITY PRIMARY KEY,
    first_name VARCHAR(255) NOT NULL,
    last_name VARCHAR(255) NOT NULL,
    email VARCHAR(255) NOT NULL,
    phone VARCHAR(20)
);

''')

ProgrammingError: ('42S01', "[42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]There is already an object named 'persons' in the database. (2714) (SQLExecDirectW)")

Note that the NOT NULL constraints are always written as column constraints.

By default, if you don’t specify the NOT NULL constraint, SQL Server will allow the column to accepts NULL. In this example, the phone column can accept NULL.

In [50]:
cursor.execute('''
INSERT INTO hr.persons
VALUES ( 'James', 'Bond', 'James.Bond@uk.com', '' )
''')

In [51]:
cursor.execute('''
SELECT * FROM hr.persons
''')


# Fetch all rows from the executed query
rows = cursor.fetchall()

# Get the column names
columns = [column[0] for column in cursor.description]

# Convert the rows into a list of dictionaries
data = [dict(zip(columns, row)) for row in rows]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)
df.head(10)

,person_id,first_name,last_name,email,phone
0,1,James,Bond,James.Bond@uk.com,
1,2,James,Bond,James.Bond@uk.com,


To add / remove NOT NULL property, use ALTER TABLE 

In [53]:
cursor.execute('''
ALTER TABLE hr.persons
ALTER COLUMN phone VARCHAR(20) NOT NULL;
''')

In [54]:
cursor.execute('''
SELECT * FROM hr.persons
''')


# Fetch all rows from the executed query
rows = cursor.fetchall()

# Get the column names
columns = [column[0] for column in cursor.description]

# Convert the rows into a list of dictionaries
data = [dict(zip(columns, row)) for row in rows]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)
df.head(10)

,person_id,first_name,last_name,email,phone
0,1,James,Bond,James.Bond@uk.com,
1,2,James,Bond,James.Bond@uk.com,


In [55]:
cursor.execute('''
INSERT INTO hr.persons
VALUES ( 'James', 'Bond', 'James.Bond@uk.com', '007' )
''')

In [56]:
cursor.execute('''
SELECT * FROM hr.persons
''')


# Fetch all rows from the executed query
rows = cursor.fetchall()

# Get the column names
columns = [column[0] for column in cursor.description]

# Convert the rows into a list of dictionaries
data = [dict(zip(columns, row)) for row in rows]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)
df.head(10)

,person_id,first_name,last_name,email,phone
0,1,James,Bond,James.Bond@uk.com,
1,2,James,Bond,James.Bond@uk.com,
2,3,James,Bond,James.Bond@uk.com,007


### UNIQUE

SQL Server UNIQUE constraints allow you to ensure that the data stored in a column, or a group of columns, is unique among the rows in a table.

In [57]:
cursor.execute('''
DROP TABLE hr.persons
''')

In [58]:
cursor.execute('''
CREATE TABLE hr.persons(
    person_id INT IDENTITY PRIMARY KEY,
    first_name VARCHAR(255) NOT NULL,
    last_name VARCHAR(255) NOT NULL,
    email VARCHAR(255),
    UNIQUE(email)
);

''')

In [59]:
cursor.execute('''
INSERT INTO hr.persons(first_name, last_name, email)
VALUES('John','Doe','j.doe@bike.stores');
''')

In [60]:
cursor.execute('''
INSERT INTO hr.persons(first_name, last_name, email)
VALUES('Jane','Doe','j.doe@bike.stores');
''')

IntegrityError: ('23000', "[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Violation of UNIQUE KEY constraint 'UQ__persons__AB6E616485DF55F9'. Cannot insert duplicate key in object 'hr.persons'. The duplicate key value is (j.doe@bike.stores). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The statement has been terminated. (3621)")

Note the error - "Violation of UNIQUE KEY constraint 'UQ__persons__AB6E616485DF55F9'. Cannot insert duplicate key in object 'hr.persons'. The duplicate key value is (j.doe@bike.stores)."

The following are the benefits of assigning a UNIQUE constraint a specific name:

    It easier to classify the error message.
    You can reference the constraint name when you want to modify it.

### CHECK

In [62]:
cursor.execute('''

CREATE TABLE hr.products(
    product_id INT IDENTITY PRIMARY KEY,
    product_name VARCHAR(255) NOT NULL,
    unit_price DEC(10,2) CHECK(unit_price > 0)
);
''')

As you can see, the CHECK constraint definition comes after the data type. It consists of the keyword CHECK followed by a logical expression in parentheses:

CHECK(unit_price > 0)

In [64]:
cursor.execute('''
INSERT INTO hr.products(product_name, unit_price)
VALUES ('Awesome Free Bike', 0);
''')

IntegrityError: ('23000', '[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The INSERT statement conflicted with the CHECK constraint "CK__products__unit_p__5165187F". The conflict occurred in database "hr", table "hr.products", column \'unit_price\'. (547) (SQLExecDirectW); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The statement has been terminated. (3621)')

Notice the error you get - The INSERT statement conflicted with the CHECK constraint "CK__products__unit_p__5165187F". The conflict occurred in database "hr", table "hr.products", column \'unit_price\'.

In [66]:
cursor.execute('''
INSERT INTO hr.products(product_name, unit_price)
VALUES ('Awesome Bike', 599);

''')

In [67]:
cursor.execute('''
SELECT * FROM hr.products
''')


# Fetch all rows from the executed query
rows = cursor.fetchall()

# Get the column names
columns = [column[0] for column in cursor.description]

# Convert the rows into a list of dictionaries
data = [dict(zip(columns, row)) for row in rows]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)
df.head(10)

,product_id,product_name,unit_price
0,2,Awesome Bike,599.00


We can ap[ply CHECK contraint to multiple columns

In [68]:
cursor.execute('''
CREATE TABLE hr.product_items(
    product_id INT IDENTITY PRIMARY KEY,
    product_name VARCHAR(255) NOT NULL,
    unit_price DEC(10,2) CHECK(unit_price > 0),
    discounted_price DEC(10,2) CHECK(discounted_price > 0),
    CHECK(discounted_price < unit_price)
);
''')